Building Music Recommendation System using Spotify Dataset

In an era where digital music consumption is at an all-time high, the ability to tailor music recommendations to individual preferences has become increasingly important. This project aims to develop a music recommendation system that provides personalized song suggestions to users based on their listening habits and preferences. By leveraging data-driven techniques and machine learning algorithms, this system aspires to enhance user experience by delivering music that aligns with their tastes.

In [1]:
#Import Libraries

import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/Users/theophilusagbolade/Documents/GitHub/spotify_recommendation/data.csv.zip')